import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# EDSA - Climate Change Belief Analysis 2021
----------------

### Summary
-----------------
With climate change considered a problem throughout the world many companies have taken the initiative to develope enviromentallyy friendly products and services to aim in lessening their carbon footprint. These compannies, however, often find it difficult to guage how their products and services may be percieved and therefore require a means of determining people's views on climate change. By analysing tweets about climate change the sentiment/opinion of the person writing that tweet can be captured and used to provide insight into who is anti or pro climate change and thus guide companies of who to target their products/services to. In this classification problem the aim was to accurately classify sentiments for a collection of tweets about climate change. To do this five, namely LinearSVC, Naive Bayes Multinomial, Random Forest, K-Nearest Neighbors and Logistic Regression classifier algorithms were used to build models to classify/predict tweet sentiments. Each of the models were assessed using several classification metrics, with the LinearSVC classifier displaying the best scores for the data at hand. We subsequently used a Grid Search to improve on the hyperparameters of LinearSVC in an attempt to improve its classifying ability. 


<img src = 'https://media3.giphy.com/media/k4ZItrTKDPnSU/200.gif'></img>

### Contents
---------------
1. [Introduction](#introduction)
2. [Import Packages](#import)
3. [Loading Data and Exploratory Data Analysis (EDA)](#load-data)
4. [Pre-Processing and further EDA](#preprocess)
5. [Model Building](#model-building)
    * [LinearSVC](#linearsvc)
    * [Naive Bayes](#naive-bayes)
    * [Random Forest](#random-forest)
    * [KNeighbors](#kneighbors)
    * [Logistic Regression](#logistic-regression)
6. [Model Assessment](#model-assess)
7. [Conclusion](#conclusion)

<a id="introduction"></a>
## 1. Introduction
---------------
Climate change is the phenomenon of an increasing number of greenhouse gases within the earth's atmosphere that is accompanied by major shifts in weather patterns. This is largely human-induced and is as a result of increased levels of atmospheric carbon dioxide produced by the use of fossil fuels for basic living neccesities as well as large industrial processes. The effects of climate change affect the livelihoods of both people and animals and is experienced via intense drought, storms, heat waves, rising sea levels, melting glaciers and warming oceans, Furthermore, as climate change worsens, dangerous weather events are becoming more frequent or severe.

<img src = 'https://media.giphy.com/media/hPovBcQ3c1g9W/giphy.gif'></img>

Over several years, many companies have attempted to implemt startegies around lessening their environmental impact or carbon footprint. They offer products and services that are environmentally friendly and sustainable, in line with their values and ideals. However, problems they experience include guaging how their products may be recieved based on people's views and opinions of climate change. By determining how people perceive climate change and whether or not they believe it is a real threat, these companies can improve on their market research efforts. Additionally, access to a large collection of consumer sentiments that also spans multiple demographic and geographic categories will influence company insights and their future marketing strategies allowing them to find the right target market to direct their products and efforts toward.

In the context of climate change and sustainable companies, Team_TS1 aim to provide a means for such companies to determine the views or sentiments of people towards climate change. To do this, tweets pertaining to climate change will be looked at and used to train a classification model in order to accurately classify the opinions behind those tweets, into those who believe in climate change and those who do not. This notebook details the work flow of Team_TS1 in building, training and assessing different classifier models to provide a suitable solution that can be implemented in future marketing strategies of climate concious companies. 

<a id="import"></a>
## 2. Import Packages
--------------------
To carry out EDA, model building and assessment of model performance

In [ ]:
# import comet_ml
from comet_ml import Experiment

In [1]:
# Numpy and Pandas
import numpy as np
import pandas as pd

# Visualisations
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# Scikit Learn 
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer


# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#Extra
from wordcloud import WordCloud
import pickle
import string

<a id="load-data"></a>
## 3. Loading Data and Exploratory Data Analysis (EDA)
-------------------
The data is provided in two files: train.csv and test_with_no_labels.csv and pertains to climate change tweets collected between Apr 27, 2015 and Feb 21, 2018. The train.csv will be used to train the classifier model and the test_no_labels.csv will be used to test the model's classifying accuracy.

### 3.1. Training Dataset
-----------
The training set has 15819 rows and 3 columns, sentiment, made up of integer type data, message, made up of object type data and tweetid, made up of integer type data. Each row represents one observation and each column, a feature. 

**Columns:**
* **message** : the written tweet, on climate change.
* **tweetid** : identifier of a status/message on twitter
* **sentiment** : expresses the view/opinion of climate change (Belief or Disbelief in  climate change, ranges from -1 to 2)
        - -1. Anti - Climate Change: the tweet does not believe in man-made climate change
        -  0. Neutral: the tweet neither supports nor refutes the belief of man-made climate change 
        -  1. Pro - Climate Change: the tweet supports the belief of man-made climate change
        -  2. News: the tweet links to factual news about climate change

In [3]:
# loading and reading train.csv to a Dataframe
train = pd.read_csv('train.csv')

In [ ]:
# Overview of the train_df dataset
train.info()

In [ ]:
# View first five rows of train_df
train.head()

### 3.2. Testing Dataset
------------
The testing set has 10546 rows and 2 columns; message and tweetid. The sentiment is to be predicted by classification.

In [4]:
# loading and reading test_with_no_labels.csv to a Dataframe
test = pd.read_csv('test_with_no_labels.csv')

In [ ]:
# View first five rows of test_df
test.head()

In [ ]:
# Overview of the test_df dataset
test.info()

### 3.3. EDA of Training Data
-----------
**Distribution of the Response**

The distribution of the label variable (sentiment) provides insight into the frequencies per category of the sentiment (-1 to 2) expressed by each tweet. This gives an indication of the more popular vs unpopular opinions on climate change. Given the categorical nature of this variable a countplot provides the best insight into the frequencies for each category.

In [ ]:
def change_to_words(sentiment):
    if sentiment == -1:
        return 'Anti'
    elif sentiment == 0:
        return 'Neutral'
    elif sentiment == 1:
        return 'Pro'
    else:
        return 'News'

In [ ]:
sentiments = train['sentiment'].apply(change_to_words)

In [ ]:
sns.set_style('darkgrid')
fig, ax = plt.subplots(ncols=2, 
                         nrows=1, 
                         figsize=(20, 10), 
                         dpi=100)

sns.countplot(sentiments, ax=ax[0], palette = ['green', 'royalblue', 'orange', 'red'])

labels = ['Pro', 'News', 'Neutral', 'Anti'] 

ax[1].pie(sentiments.value_counts(),
            labels =labels,
            autopct = '%1.0f%%',
            startangle = 90,
            shadow = True,
            explode = (0.0, 0.1, 0.0, 0.0), 
            colors = ['green', 'royalblue', 'orange', 'red'], 
            textprops = {'fontsize': 15})

fig.suptitle('Sentiment Distribution', fontsize = 30)
plt.legend(title = 'Sentiments')
plt.show()

The distribution of sentiments shows a clear difference in the frequencies observed for each sentiment, with the 'Pro' climate change tweets making up majority of the opinions expressed within this data set at over 8000 counts. 'Anti' climate changes tweets make up the lowest number of opinons with just over 1000 counts and with more people having a neutral response towards climate change than dibelieving in it. The number of News/factual tweets is higher than both that of 'Anti' and 'Neutral' views over 3000 views. 

The following table of value counts further describes the difference in frequency between each sentiment.

In [ ]:
# Total number of observations for each sentiment
sentiment_counts = pd.DataFrame(train['sentiment'].value_counts())
sentiment_counts['View'] = ['Pro', 'News/Factual', 'Neutral', 'Anti']
sentiment_counts.sort_index(axis = 1)

<a id="preprocess"></a>
## 4. Pre-Processing and further EDA
--------------
#### <u>Prodecure for Pre-Processing</u>
* Hashtag Extraction
* Data Cleaning
* Tokenisation
* Lemmatization

In [7]:
# Make a copy of the train dataset to analyse
df_train = train.copy()

### 4.1 Hashtag Extraction (#)
--------------
Hashtags are used to categorize tweets, so we will be extracting words starting with a `#` to see how many times they occur.

In [8]:
def extract_hashtags(post):
    post_ = [word.replace('#', '') for word in post.split() if word.startswith('#')]
    return post_

In [9]:
df_train['hashtags'] = df_train['message'].apply(extract_hashtags)

In [ ]:
df_train.head()

After all hashtags has been extracted, we can go ahead and clean the data.

### 4.2 Data Cleaning
------------

**Noisy information to remove:**
* URL's
* mentions
* stopwords
* punctuations
* numbers and alphanumeric words
* extra whitespaces

Making everything lowercase.

In [10]:
# Remove URL's
def remove_links(df, column):
    regex_pattern = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+'
    df[column] = df[column].replace(to_replace = regex_pattern, 
                                    value = '', 
                                    regex = True)
    
# Remove mentions
def remove_mentions(post):
    post_ = ' '.join([word for word in post.split() if not word.startswith('@')])
    return post_

# Remove stopwords
stop_words = stopwords.words('english')
def remove_stopwords(post):
    post = ' '.join([word for word in post.split() if word not in stop_words])
    return post

# Remove punctuations
def remove_punctuation(post):
    post = ''.join([x for x in post if x not in string.punctuation])
    return post

# Remove numbers and alphanumeric words
def return_alpha(post):
    post_ = ' '.join([word for word in post.split() if word.isalpha()])
    return post_

# Remove extra spaces
def remove_spaces(df, column):
    regex_p = r'\s\s+'
    df[column] = df[column].replace(to_replace = regex_p, 
                                    value = '', 
                                    regex = True)
# Making everything lowercase
df_train['message'] = df_train['message'].str.lower()

In [11]:
remove_links(df_train, 'message')
df_train['message'] = df_train['message'].apply(remove_mentions)
df_train['message'] = df_train['message'].apply(remove_stopwords)
df_train['message'] = df_train['message'].apply(remove_punctuation)
df_train['message'] = df_train['message'].apply(return_alpha)
remove_spaces(df_train, 'message')

In [ ]:
df_train.head()

The tweets needs to be tokenised to help get a better understanding of the context. Let's go ahead and tokenise of tweets. 

### 4.3 Tokenisation
------------
Tokenization is a way of separating a piece of text into smaller units. Tokens can be either words, characters or subwords.

In [12]:
def tokenize(post):
    post_ = word_tokenize(post)
    return post_

In [13]:
df_train['tokens'] = df_train['message'].apply(tokenize)

In [ ]:
df_train.head()

Next, we will be lemmatizing all the words to their base form.

### 4.4 Lemmatization
------
Lemmatizing words to their base form will ensure that words with similar meaning be represented in the same way. 

Stemming is not an option because it looks for similarity in words and often returns words that have no meaning/value. 

In [14]:
lemmatizer = WordNetLemmatizer()

In [15]:
def lemmatize_(post):
    row = []
    for word in post:
        lem = lemmatizer.lemmatize(word)
        row.append(lem)
    post_ = ' '.join(row)
    return post_

In [16]:
df_train['lemmatized'] = df_train['tokens'].apply(lemmatize_)

In [ ]:
df_train.head()

Now that the pre-processing is done, we can visual and find trends in our data.

### 4.5 Further EDA
------
We will be looking at:
* Most frequently used words
* Hashtags

#### Most Frequently used words

In [17]:
sns.set_style('darkgrid')

In [18]:
# Put all words in a single string so it can be processed by the wordcloud
def change_to_string(sentiment):
    words = []
    for post in df_train[df_train['sentiment'] == sentiment]['lemmatized']:
        for word in post.split():
            words.append(word)
    text = ' '.join(words)
    return text

In [19]:
anti_climate = change_to_string(-1)
pro_climate = change_to_string(1)
neutral_climate = change_to_string(0)
news_climate = change_to_string(2)

In [20]:
anti_cloud = WordCloud(
    background_color ='white', 
    max_font_size = 130, 
    max_words = 20, 
    colormap = "Reds").generate(anti_climate)

pro_cloud = WordCloud(
    background_color ='white', 
    max_font_size = 130, 
    max_words = 20, 
    colormap = "Greens").generate(pro_climate)

neutral_cloud = WordCloud( 
    background_color ='white', 
    max_font_size = 130, 
    max_words = 20, 
    colormap = "Oranges").generate(neutral_climate)

news_cloud = WordCloud(
    background_color ='white', 
    max_font_size = 130, 
    max_words = 20, 
    colormap = "Blues").generate(news_climate)

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(35, 25))
ax[0, 0].imshow(pro_cloud, interpolation="bilinear")
ax[0, 1].imshow(anti_cloud, interpolation="bilinear")
ax[1, 0].imshow(neutral_cloud, interpolation="bilinear")
ax[1, 1].imshow(news_cloud, interpolation="bilinear")

ax[0, 0].set_title('\nPro climate change\n', fontsize = 40, fontweight = 'bold')
ax[0, 1].set_title('\nAnti climate change\n', fontsize = 40, fontweight = 'bold')
ax[1, 0].set_title('\nNeutral\n', fontsize = 40, fontweight = 'bold')
ax[1, 1].set_title('\nNews\n', fontsize = 40, fontweight = 'bold')

ax[0, 0].axis("off")
ax[0, 1].axis("off")
ax[1, 0].axis("off")
ax[1, 1].axis("off")
plt.show()

#### Hashtags

In [21]:
# Get hashtags for a specific sentiment
def get_hashtags(sentiment):
    hashtags = []

    for hashtag in df_train[df_train['sentiment'] == sentiment]['hashtags']:
        for word in hashtag:
            hashtags.append(word.lower())
            
    return pd.Series(hashtags)

In [22]:
anti = get_hashtags(-1).value_counts()
neutral = get_hashtags(0).value_counts()
pro = get_hashtags(1).value_counts()
news = get_hashtags(2).value_counts()

In [ ]:
sns.barplot(
    x = anti[:15],
    y = list(anti.index)[:15],
    palette = 'Reds_d'
)

plt.title('Anti Climate Change Hashtags', fontsize = 15, fontweight='bold')


In [ ]:
sns.barplot(
    x = neutral[:15],
    y = list(neutral.index)[:15], 
    palette = 'Oranges_d'
)
plt.title('Neutral Hashtags', fontsize = 15, fontweight='bold')

In [ ]:
sns.barplot(
    x = pro[:15],
    y = list(pro.index)[:15], 
    palette = 'Greens_d'
)
plt.title('Pro Climate Change Hashtags', fontsize = 15, fontweight='bold')

In [ ]:
sns.barplot(
    x = news[:15],
    y = list(news.index)[:15], 
    palette = 'Blues_d'
)
plt.title('News Hashtags', fontsize = 15, fontweight='bold')

<a id="model-building"></a>
## 5. Model Building 
---
The model building section is only for model building not for comparison between each models results, that will be covered in the next section.
<br></br>
#### <b><u>Models being used:</u></b>
<ul>
    <li>LInearSVC</li>
    <li>Naive Bayes</li>
    <li>Random Forest</li>
    <li>KNeighbors</li>
    <li>Logistic Regression</li>
</ul>

#### Procedure for the model building section
---
<ol>
    <li>Assign the independent variable and target variable to variables X and y, respectively</li>
    <li>Split the data into a train and validation set</li>
    <li>Create a pipeline to set workflow (contains vectorizer and model)</li>
    <li>Fit the model with the train set</li>
    
</ol>
<b><u>Note:</u></b> Step 3 to Step 4 will be repeated for all models. All models are run on their default hyperparameter setting to establish which model works best at predicting the data at hand.

Each model was built through a Pipeline with a Tfidf-Vectorizer (Term Frequency Inverse Document Frequency). The algorithm for this vectorizer transforms text into a representation of numbers that reflect how significant a word is to a record in a collection or corpus. 

The output of the vectorizer can be controlled via several hyperparameters. Here the min_df, max_df and ngram_range hyperparameters were adjusted for each classifier to increase prediction capabilities of each model. Min_df and max_df stands for the frequency threshold of the words to be ignored from the vectorizer output if they appear less than or more than the values set for each respective hyperparameter. The ngram_range creates a sequence of N-words in a sentence for each word in a record, where N is an integer that stands for the number of words in the sequence.


In [23]:
# Declaring the X and y variables
X = train['message']
y = train['sentiment']

Splitting the data into 75% train and 25% test with a `random_state` of 42

In [24]:
# splitting training dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 42)

After splitting our data, we can begin to use it to train our models.

In [ ]:
# setting up of Tfidf vectorizer to run with each model in a Pipeline
tfidf_v = TfidfVectorizer(ngram_range = (1, 2), min_df = 2, max_df = 0.8)

<a id="linearsvc"></a>
### 5.1 LinearSVC
----------

Support Vector Machine (SVM) models are most commonly associated with classification tasks and are found to be among the more accurately classifying algorithms. The SVM algorithm plots each data point in an N-dimensional space, with N being the number of features present within the dataset. There are a number of hyperplanes that exist within the dimensional space with each hyperplane separating each class of data points to a certain extent. The algorithm will subsequently look for the ‘best’ hyperplane which would have the maximum distance between the classes of data points (maximum margin), distinctly separating each class/label.

In this classification problem, a LinearSVC model was used to classify the tweet sentiments. The LinearSVC model uses a liblinear estimator which penalizes the intercept and minimizes the squared hinge loss function, compared to the liblinear estimators of a SVC() model that do not penalize the intercept. Liblinear estimators are optimized for a linear case and are able to converge quicker on larger amounts of data and is therefore quick in resolve=ing the problem. LinearSVC() uses a One-vs-All multiclass-reduction by splitting the multiclass dataset into multiple binary classification problems. 

In [25]:
lsvc_ = Pipeline([
    ('tfidf', tfidf_v),
    ('lsvc', LinearSVC())
])

In [26]:
lsvc_.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('lsvc', LinearSVC())])

<a id="naive-bayes"></a>
### 5.2 Naive Bayes
-----------

The Naive Bayes Classifier is among the faster computing classification algorithms, is mostly used in text classification/natural language processing problems and in cases where datasets have multiple classes. This algorithm uses Bayes theorem to predict the class of unknown data points using conditional probability, which is a measure of the probability of an event occurring given that another event has occurred. It assumes independence between the features of the data set and uses the Naive Bayesian equation to calculate the posterior probability for each class, with the highest posterior probability being the outcome of prediction. In the case of this classification problem the Multinomial variant of the Naive Bayes classifier built for sentiment prediction. This classifier assumes a feature vector where a given term is represented by the number/frequency of times it appears, the algorithm then determines the probability for each term and the predicted outcome is based on the highest probability. The main hyperparameter associated with the Multiomial Naive Bayes classifier is alpha (Laplace smoothing factor) that prevents the model from setting null probabilities when the frequency is zero while larger values of alpha assign higher probabilities to the missing features.

In [27]:
mnb_ = Pipeline([
    ('tfidf', tfidf_v),
    ('clf', MultinomialNB())
])

In [28]:
mnb_.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('clf', MultinomialNB())])

<a id="random-forest"></a>
### 5.3 Random Forest
------------

Next, we built a model using the Random Forest classifier. This classifier creates a set of decision trees from randomly selected subsets of the training set. It then aggregates the votes from different decision trees to decide the final class of the test object. 

In [29]:
rf_ = Pipeline([
    ('tfidf', tfidf_v),
    ('rf', RandomForestClassifier())
])

In [30]:
rf_.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('rf', RandomForestClassifier())])

<a id="kneighbors"></a>
### 5.4 KNeighbors
------------

The KNeighborsClassifier uses the K Nearest Neighbors (KNN) algorithm to classify data. This algorithm is non-parametric and works by classifying the query point as the class of the majority of representatives among the nearest neighbors of the query/test data point (i.e if most of the query point’s neighbors are Class x, then the query point will be classified as Class x). 

In [31]:
knn_ = Pipeline([
    ('tfidf', tfidf_v),
    ('knn', KNeighborsClassifier())
])

In [32]:
knn_.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('knn', KNeighborsClassifier())])

<a id="logistic-regression"></a>
### 5.5 Logistic Regression
------------

Logistic Regression is typically used for classification problems where there are two outcomes (binary). The model measures the relationship between the dependent variable (label) and one or more independent variables (our features), by estimating the probability of the first class (class 1). These probabilities are then transformed into binary values, by the logistic/sigmoid function, in order to actually make a prediction. This function is an S-shaped curve that takes any real-valued number and maps it into a value between the range of 0 and 1.

In [33]:
lr_ = Pipeline([
    ('tfidf', tfidf_v), 
    ('lr', LogisticRegression())
])

In [34]:
lr_.fit(X_train, y_train)

C:\Users\henab\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Pipeline(steps=[('tfidf',
                 TfidfVectorizer(max_df=0.8, min_df=2, ngram_range=(1, 2))),
                ('lr', LogisticRegression())])

All the models has been fitted, now we move to the Model Assessment section.

<a id="model-assess"></a>
## 6. Model Assessment 
--------------
We will be comparing the performance of the models to see which one had a best performance.
<br></br>
- We will compare each model's `precision`, `recall` and `f1` scores to see which one performed the best

The Precision score refers to the percentage of predictions that are correct (i.e out of all points classified as class one how many were correctly classified as class one). Recall scores reflect the fraction of each label that was correctly identified (i.e out of all true class ones how many were correctly predicted as class one). The f-1 score is a combination of both precision and recall, as a weighted harmonic mean and is useful for imbalanced classification. It falls between a range of 0 and 1.00, with a value of 1.00 indicating a model with perfect precision and recall and values close to zero suggesting a model with poor precision and recall. This score is also commonly used to compare classifier models. 

In [35]:
lsvc_y = lsvc_.predict(X_test)
mnb_y = mnb_.predict(X_test)
rf_y = rf_.predict(X_test)
knn_y = knn_.predict(X_test)
lr_y = lr_.predict(X_test)

In [36]:
metrics = {
    'precision': [
        precision_score(y_test, lsvc_y, average = 'weighted'),
        precision_score(y_test, mnb_y, average = 'weighted'),
        precision_score(y_test, rf_y, average = 'weighted'),
        precision_score(y_test, knn_y, average = 'weighted'),
        precision_score(y_test, lr_y, average = 'weighted')
    ],
    'recall': [
        recall_score(y_test, lsvc_y, average = 'weighted'),
        recall_score(y_test, mnb_y, average = 'weighted'),
        recall_score(y_test, rf_y, average = 'weighted'),
        recall_score(y_test, knn_y, average = 'weighted'),
        recall_score(y_test, lr_y, average = 'weighted')
    ],
    'f1': [
        f1_score(y_test, lsvc_y, average = 'weighted'),
        f1_score(y_test, mnb_y, average = 'weighted'),
        f1_score(y_test, rf_y, average = 'weighted'),
        f1_score(y_test, knn_y, average = 'weighted'),
        f1_score(y_test, lr_y, average = 'weighted')
    ]
}

In [37]:
pd.DataFrame(metrics, index = [
    'LinearSVC', 
    'Naive Bayes', 
    'Random Forest', 
    'KNeighbors', 
    'Logistic Regression'])

,precision,recall,f1
LinearSVC,0.760004,0.765866,0.757522
Naive Bayes,0.772611,0.680657,0.612546
Random Forest,0.720970,0.712769,0.686340
KNeighbors,0.656042,0.654614,0.653468
Logistic Regression,0.748897,0.749178,0.728501


**Fairly high (> 0.5) precision, recall and f1 scores are displayed for all of the models tested in sentiment classification. Of these five, LinearSVC displays the best set of scores across all the metrics.**
**In terms of f1 scores close second in terms of model performance is the Logistic Regression model,**

UNFINISHED

As we can observe from above, `LinearSVC` performed the best when running the data on it's default hyperparameters. Therefore we will tune the LinearSVC model and see if it will perform better.

<b>Procedure:</b>
* Create a dictionary with hyperparameters to run a Grid Search
* Pass the model and hyperparameters into a Cross Validation algorithm
* Observe best parameters
* Observe best score
* Observe classification report and confusion matrix


In an attempt to improve the prediction accuracy of the LinearSVC() classifier, the vectorizing, model building and model tuning was applied to each model using a Grid Search (GridSearchCV), in order to find the optimal hyperparameters that performs best with the input data for each classifier model. GridSearchCV tries all the combinations of the values passed in the dictionary and evaluates the model for each combination using the Cross-Validation method using a 'fit and score' method. K-folds of k = 5 are used for cross validation for each of the models. The data set is divided into k number of subsets/folds, the model is then trained on k-1 folds and the remaining fold is used to test the model's effectiveness. The model will iterate through each fold until all folds has served as a test set and the average k recorded accuracy (cross-validation accuracy) is used as the performance metric for the model.

The LinearSVC() classifier consists of several tunable hyperparameters that can be used to improve on it’s classifying capabilities, here the hyperparameters C and class_weight were set during model building. The C hyperparameter controls the penalty strength, where small C values increase the regularization strength which creates simple models that under-fits the data and big C values lowers the regularization strength which increases the model’s complexity resulting in overfitting of the data. The class_weight hyperparameter handles a data imbalance, with ‘balanced’ meaning classes will be automatically weighted inversely proportional to how frequently they appear in the data.

In [38]:
lsvc_params = {
    'lsvc__C': [1, 0.5],
    'lsvc__max_iter': [1000, 2000], 
    'lsvc__class_weight': ['balanced']
}

In [39]:
lsvc_cv = GridSearchCV(
    lsvc_,
    param_grid = lsvc_params,
    scoring = 'f1_weighted',
    cv = 20
)

In [40]:
lsvc_cv.fit(X_train, y_train)

GridSearchCV(cv=20,
             estimator=Pipeline(steps=[('tfidf',
                                        TfidfVectorizer(max_df=0.8, min_df=2,
                                                        ngram_range=(1, 2))),
                                       ('lsvc', LinearSVC())]),
             param_grid={'lsvc__C': [1, 0.5],
                         'lsvc__class_weight': ['balanced'],
                         'lsvc__max_iter': [1000, 2000]},
             scoring='f1_weighted')

In [41]:
lsvc_cv.best_params_

{'lsvc__C': 0.5, 'lsvc__class_weight': 'balanced', 'lsvc__max_iter': 1000}

In [ ]:
lsvc_cv.best_score_

In [ ]:
y_pred = lsvc_cv.predict(X_test)

A classification report displays several important classification metrics for each label being predicted by the model.  Precision, recall, f1, and support scores are all included in this report. The support score references the true number of instances of the class in the specified dataset. The classification report further includes the accuracy, macro and weighted average scores for each metric. The accuracy score relflects the overall performance of model as a ratio of correctly predicted observation to the total observations, however this metric gives the best insight into symmetric data One may think that, if we have high accuracy then our model is best. Yes, accuracy is a great measure but only when you have symmetric datasets, therefore, other parameters need to be evaluated as well in order to determine the performance of your model.The macro average score refers to the unweighted mean of the metric for each label to being calculated independently to find the unweighted mean. Finally the weighted average score is the metric calculated for each label/class, and finds their average weighted support.

In [ ]:
print(classification_report(y_test, y_pred))

The confusion matrix simply depicts the number of correctly classified and incorrectly classified labels.  The diagonal elements of the matrix represent the number ofdata points for which the predicted label is equal to the true label, while off-diagonal elements are those that are mislabeled by the classifier. Higher diagonal values are indicative of many correctly predicted data points. 

In [ ]:
sns.heatmap(
    confusion_matrix(y_test, y_pred), 
    cmap = "Blues_r", 
    xticklabels = lsvc_cv.classes_, 
    yticklabels = lsvc_cv.classes_, 
    vmin = 0, 
    vmax = 3000, 
    annot = True, 
    fmt = 'g'
)

plt.ylabel('True values')
plt.xlabel('Predicted')
plt.show()

After hyperparameter tuning, we can predict sentiments for the test dataset.

#### Test Set
No pre-processing is required for the test set as everythin

In [ ]:
y_pred_ = lsvc_cv.predict(test['message'])

In [ ]:
submission = pd.DataFrame({'tweetid': test['tweetid'], 
                           'sentiment': y_pred_})

In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv', index = False)

<a id="conclusion"></a>
## 7. Conclusion
--------
Of the five models tested in sentiment analysis and classification of climate change tweets, the model with the best results (f1 score) was found to be the LinearSVC(). This is likely due to the LinearSVC() model looking at the interactions between the word frequencies of the vectorizer output, as well as being better suited to unstructured and semi-structured data like text. The best hyperparameters of LinearSVC() were a C value of 0.5 and a max_iter value of 2000, with a class_weight set to 'balanced'. This model's sentiment predictions scored an f1 score of 0.74810 on Kaggle agaisnt the true sentiments of the test data set, as this score is closer to 1 than 0, it can be suggested that the chosen 'best' model is higher in its precision and recall and is thus fairly effective in predicting/classifying tweet sentiments. This model can therefore be suggested as an approach companies can take to determine those who are anti- or pro- climate change and use this information to establish the developement and marketing of their climate friendly products and services. 

While all tested model's displayed decent f1 scores, future considerations for sentiment classification of this nature can include assessing what classifier alogorithms are best suited to the data at hand, as well as, running a grid search on all tested models to find optimal hyperparameters that improve on classififcation metric scores. More care can also be taken in the cleaning up of tweet datasets which in turn may also improve metric scores.